# Previsão do custo pessoal médico

O objetivo desse projeto é testar diferentes regressões para a mesma base de dados, com o intuito de visualizar as funcionalidades do ML Flow como ferramenta de tracking de experimentos de machine learning.

In [1]:
import os
import pandas as pd

from sklearn.model_selection import train_test_split

import mlflow
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

import numpy as np
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Selecionando o diretório e as pastas do projeto

rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')

# 01) Importanto e transformando os dados

In [4]:
csv_path = os.path.join(dataPath, 'insurance.csv')
dados = pd.read_csv(csv_path)

In [5]:
# Visualizando e verificando os tipos de variáveis presentes
dados.info()
dados.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [6]:
# Vereficando campos vazios

dados.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

Temos 3 variáveis (sex, smoker e region) que vão precisam passar pelo processo de enconding para se transformarem em variáveis numéricas.

In [7]:
print("Valores únics para variável sex:", list(dados['sex'].unique()))
print("Valores únics para variável smoker:", list(dados['smoker'].unique()))
print("Valores únics para variável region:", list(dados['region'].unique()))

Valores únics para variável sex: ['female', 'male']
Valores únics para variável smoker: ['yes', 'no']
Valores únics para variável region: ['southwest', 'southeast', 'northwest', 'northeast']


Portanto:

- sex -> female = 1 / male = 0
- smoker -> yes = 1 / no = 0
- region -> será retirada para facilitar a regressão

In [8]:
# Fazendo o enconding das variáveis binárias

map_sex = {'female': 1, 'male': 0}
map_smoker = {'yes': 1, 'no': 0}

dados['sex'] = dados['sex'].replace(map_sex)
dados['smoker'] = dados['smoker'].replace(map_smoker)

In [9]:
df = dados.drop(columns=['region'])

In [10]:
df.head()

,age,sex,bmi,children,smoker,charges
0,19,1,27.900,0,1,16884.92400
1,18,0,33.770,1,0,1725.55230
2,28,0,33.000,3,0,4449.46200
3,33,0,22.705,0,0,21984.47061
4,32,0,28.880,0,0,3866.85520


# 02) Construindo os métodos e preparando os dados para treino e teste

In [11]:
seed = 1337 # Fixando a semente para os métodos

In [12]:
# Separando variáveis explicativas da target
x_true = df.drop(columns=['charges'])
y_true = df['charges']

In [13]:
# Separação da base de dados em treino e teste para cada normalização

X_train, X_test, y_train, y_test = train_test_split(x_true, y_true, test_size=0.3, random_state= seed)

In [14]:
# Construindo função que irá rodar o experimento no mlflow:

def realiza_experimento(id_experimento, modelo, nome_experimento):

    with mlflow.start_run(experiment_id=id_experimento):

        mlflow.set_tag("mlflow.runName", nome_experimento)

        regressor = modelo
        regressor.fit(X_train, y_train)
        mlflow.sklearn.log_model(regressor, str(modelo))
        
        predict_test = regressor.predict(X_test)
        mse = mean_squared_error(y_test, predict_test)
        rmse =  np.sqrt(mse)
        r2 = r2_score(y_test, predict_test)

        mlflow.log_metric('mse',mse)
        mlflow.log_metric('rmse',rmse)
        mlflow.log_metric('r2',r2)
    
    return print(f"Experimento realizado: {nome_experimento}. \nMétricas: \n r2: {r2} \n mse: {mse} \n rmse: {rmse}")

# 03) Realizando os experimentos via mlflow

In [15]:
mlflow.set_experiment("Previsão de custos médicos pessoais")

2023/05/26 12:11:24 INFO mlflow.tracking.fluent: Experiment with name 'Previsão de custos médicos pessoais' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/gusta/Desktop/code/previsao-custo-medico/mlruns/826251267040691654', creation_time=1685113884750, experiment_id='826251267040691654', last_update_time=1685113884750, lifecycle_stage='active', name='Previsão de custos médicos pessoais', tags={}>

**Experimentos para base não normalizada**

In [16]:
realiza_experimento(
    id_experimento= '826251267040691654',
    modelo= LinearRegression(), 
    nome_experimento= "Linear"
    )

2023/05/26 12:11:40 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



Experimento realizado: Linear. 
Métricas: 
 r2: 0.6971366145626732 
 mse: 43299590.310675114 
 rmse: 6580.242420357712


In [17]:
realiza_experimento(
    id_experimento= '826251267040691654',
    modelo= DecisionTreeRegressor(random_state= seed), 
    nome_experimento= "Decision Tree"
    )

Experimento realizado: Decision Tree. 
Métricas: 
 r2: 0.7005110817692516 
 mse: 42817151.512896135 
 rmse: 6543.481604841274


In [18]:
realiza_experimento(
    id_experimento= '826251267040691654',
    modelo= GradientBoostingRegressor(random_state= seed), 
    nome_experimento= "Gradient Boosting"
    )

Experimento realizado: Gradient Boosting. 
Métricas: 
 r2: 0.8261889444619763 
 mse: 24849314.43725081 
 rmse: 4984.908668897637
